# 결측치 데이터
- 코로나 데이터셋
  + 데이터는 https://github.com/owid/covid-19-data/tree/master/public/data/
  + 매일 업데이트 된다는 점에 유의한다. 
- 2022년 1월 9일자로 데이터를 다운로드 받았다. 
  + 결괏값은 조금 다를 수 있다. 
  

## 구글 드라이브 연동 코드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 제공된 데이터를 불러온다. 

In [ ]:
import pandas as pd 
import numpy as np 

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/2022/lecture notes/PART_I/Chapter 3. pandas/data/owid-covid-data.csv"
df = pd.read_csv(DATA_PATH)
df

FileNotFoundError: ignored

In [1]:
import pandas as pd 
import numpy as np

url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
df = pd.read_csv(url)
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153625,ZWE,Africa,Zimbabwe,2022-01-06,220178.0,1121.0,1207.143,5108.0,16.0,15.857,14588.889,74.277,79.985,338.454,1.060,1.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6048.0,1740966.0,115.356,0.401,5698.0,0.378,0.2119,4.7,tests performed,7323933.0,4154155.0,3169778.0,NaN,13877.0,11002.0,48.53,27.53,21.00,NaN,729.0,5101.0,0.034,52.78,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
153626,ZWE,Africa,Zimbabwe,2022-01-07,221282.0,1104.0,1146.286,5136.0,28.0,18.857,14662.039,73.151,75.952,340.309,1.855,1.249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6723.0,1747689.0,115.801,0.445,5780.0,0.383,0.1983,5.0,tests performed,7338669.0,4160978.0,3177691.0,NaN,1473

## 결측치 확인
- 코드북은 다음 URL에서 확인할 수 있다. 
  + 참조: https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-codebook.csv

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153630 entries, 0 to 153629
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    153630 non-null  object 
 1   continent                                   144376 non-null  object 
 2   location                                    153630 non-null  object 
 3   date                                        153630 non-null  object 
 4   total_cases                                 150978 non-null  float64
 5   new_cases                                   150903 non-null  float64
 6   new_cases_smoothed                          149752 non-null  float64
 7   total_deaths                                133709 non-null  float64
 8   new_deaths                                  133870 non-null  float64
 9   new_deaths_smoothed                         133738 non-null  float64
 

- 데이터를 크게 두개로 구분하여 분류한다. 
  + Covid Case & Demographic Columns

In [3]:
case_vars = ["location", "total_cases", "total_deaths", "total_cases_per_million", "total_deaths_per_million"]
demo_vars = ["population", "population_density", "median_age", "gdp_per_capita", "hospital_beds_per_thousand"]

df[demo_vars].isnull().sum(axis = 0)


population                     1021
population_density            16886
median_age                    26023
gdp_per_capita                25608
hospital_beds_per_thousand    39250
dtype: int64

In [4]:
df[case_vars].isnull().sum(axis = 0)

location                        0
total_cases                  2652
total_deaths                19921
total_cases_per_million      3356
total_deaths_per_million    20612
dtype: int64

- 이번에는 행 방향으로 발생한 결측치를 확인한다. 

In [5]:
demovars_misscnt = df[demo_vars].isnull().sum(axis=1)
demovars_misscnt.value_counts()

0    108297
1     18005
4     13833
2      7076
3      5398
5      1021
dtype: int64

- 인구 통계 데이터가 4가지 이상 누락된 국가를 나열해보자. 

In [6]:
df.loc[demovars_misscnt >= 4, ['location'] + demo_vars]['location'].value_counts()

Lower middle income                720
High income                        720
North America                      720
Asia                               720
Upper middle income                720
European Union                     719
Europe                             719
Oceania                            717
International                      704
Africa                             698
South America                      689
Low income                         688
Vatican                            676
Montserrat                         664
Anguilla                           654
Bonaire Sint Eustatius and Saba    649
Falkland Islands                   647
Saint Pierre and Miquelon          646
Saint Helena                       491
Wallis and Futuna                  449
Guernsey                           352
Northern Cyprus                    317
Jersey                             298
Cook Islands                       235
Tokelau                            114
Pitcairn                 

- 이러한 국가들은 제거를 하는 것이 좋다. 

In [7]:
miss_locations = list(df.loc[demovars_misscnt >= 4, ['location'] + demo_vars]['location'].value_counts().index)
miss_locations[0:5]

['Lower middle income',
 'High income',
 'North America',
 'Asia',
 'Upper middle income']

- 기존 15만개 데이터에서 13만개 데이터로 줄임

In [8]:
df2 = df[~df['location'].isin(miss_locations)].reset_index(drop = False) 
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138776 entries, 0 to 138775
Data columns (total 68 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   index                                       138776 non-null  int64  
 1   iso_code                                    138776 non-null  object 
 2   continent                                   138056 non-null  object 
 3   location                                    138776 non-null  object 
 4   date                                        138776 non-null  object 
 5   total_cases                                 137340 non-null  float64
 6   new_cases                                   137258 non-null  float64
 7   new_cases_smoothed                          136223 non-null  float64
 8   total_deaths                                123871 non-null  float64
 9   new_deaths                                  123832 non-null  float64
 

# 행 필터
- 날짜를 기준으로 코로나 초기부터 6월 30일까지 데이터만 추출한다. 

In [9]:
df3 = df2.loc[df2['date'].between("2020-02-24", "2020-06-30")]
df3

,index,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138212,153066,ZWE,Africa,Zimbabwe,2020-06-26,561.0,10.0,11.714,6.0,0.0,0.286,37.172,0.663,0.776,0.398,0.000,0.019,1.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,489.0,29177.0,1.933,0.032,437.0,0.029,0.0268,37.3,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.37,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
138213,153067,ZWE,Africa,Zimbabwe,2020-06-27,567.0,6.0,12.571,6.0,0.0,0.286,37.569,0.398,0.833,0.398,0.000,0.019,1.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464.0,29641.0,1.964,0.031,456.0,0.030,0.0276,36.3,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.37,15092171.0,42.729,19.6,2.822,1.8

- 이번에 다시 결측치가 존재하는 행을 추출한다. 

In [10]:
demovars_misscnt = df3[demo_vars].isnull().sum(axis=1)
demovars_misscnt.value_counts()

0    18314
1     2595
2     1076
3      887
dtype: int64

- 결측치가 3개인 데이터를 조회한다. 

In [11]:
df3.loc[demovars_misscnt >= 3, ['location'] + demo_vars]['location'].value_counts()

Taiwan                      128
Andorra                     121
Faeroe Islands              119
Gibraltar                   119
Greenland                   107
Isle of Man                 103
British Virgin Islands       95
Turks and Caicos Islands     95
Name: location, dtype: int64

In [12]:
df3[df3['location'] == "Taiwan"][['date'] + demo_vars]

,date,population,population_density,median_age,gdp_per_capita,hospital_beds_per_thousand
122107,2020-02-24,23855008.0,NaN,42.2,NaN,NaN
122108,2020-02-25,23855008.0,NaN,42.2,NaN,NaN
122109,2020-02-26,23855008.0,NaN,42.2,NaN,NaN
122110,2020-02-27,23855008.0,NaN,42.2,NaN,NaN
122111,2020-02-28,23855008.0,NaN,42.2,NaN,NaN
...,...,...,...,...,...,...
122230,2020-06-26,23855008.0,NaN,42.2,NaN,NaN
122231,2020-06-27,23855008.0,NaN,42.2,NaN,NaN
122232,2020-06-28,23855008.0,NaN,42.2,NaN,NaN
122233,2020-06-29,23855008.0,NaN,42.2,NaN,NaN


- 이번에는 코로나 사례 데이터의 결측치를 확인한다. 

In [13]:
totvars_misscnt = df3[case_vars].isnull().sum(axis=1)
totvars_misscnt.value_counts()

0    17139
2     5555
4      178
dtype: int64

In [14]:
df3.loc[totvars_misscnt == 4, case_vars]['location'].value_counts()

Fiji            24
Libya           20
Guatemala       19
Albania         13
Peru            11
Slovenia        10
South Africa    10
Serbia           9
Argentina        8
Morocco          7
Kenya            7
Bangladesh       5
Luxembourg       5
Malawi           4
Mexico           4
Estonia          3
Senegal          3
Denmark          3
Cyprus           2
Togo             2
Latvia           2
Malta            2
Paraguay         1
Hungary          1
Iceland          1
Portugal         1
Panama           1
Name: location, dtype: int64

- Iceland 국가만 확인한다. 

In [15]:
df3[df3['location'] == "Portugal"]

,index,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
101190,112259,PRT,Europe,Portugal,2020-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,0.002,NaN,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.56,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,-1942.6,-7.86,-8.18,-191.051801
101191,112260,PRT,Europe,Portugal,2020-03-02,2.0,2.0,NaN,NaN,NaN,NaN,0.197,0.197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,70.0,0.007,0.004,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.56,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,NaN,NaN,NaN,NaN
101192,112261,PRT,Europe,Portugal,2020-03-03,2.0,0.0,NaN,NaN,NaN,NaN,0.197,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,131.0,0.013,0.006,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.56,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,NaN,NaN,NaN,NaN
101193,112262,PRT,Europe,Portugal,2020-03-04,5.0,3.0,NaN,NaN,NaN,NaN,0.492,0.295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,171.0,0.017,0.004,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.56,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,NaN,NaN,NaN,NaN
101194,112263,PRT,Europe,Portugal,2020-03-05,8.0,3.0,NaN,NaN,NaN,NaN,0.787,0.295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.885,NaN,NaN,NaN,NaN,80.0,251.0,0.025,0.008,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.56,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101307,112376,PRT,Europe,Portugal,2020-06-26,40866.0,451.0,343.143,1555.0,6.0,4.000,4019.110,44.355,33.748,152.932,0.590,0.393,1.04,67.0,6.589,457.0,44.945,NaN,NaN,NaN,NaN,15766.0,1168133.0,114.884,1.551,12409.0,1.220,0.0277,36.2,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.65,10167923.0,112.371,46.2,21.502,14.924,27936.896,0.5,127.842,9.85,16.3,30.0,NaN,3.39,82.05,0.864,NaN,NaN,NaN,NaN
101308,112377,PRT,Europe,Portugal,2020-06-27,41189.0,323.0,335.429,1561.0,6.0,4.714,4050.876,31.767,32.989

## 결측치 채우기 

In [16]:
df4 = df3.copy()
df4[case_vars].isnull().sum(axis = 0)

location                       0
total_cases                  179
total_deaths                5732
total_cases_per_million      179
total_deaths_per_million    5732
dtype: int64

In [17]:
df4[case_vars].fillna(0)

,location,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million
0,Afghanistan,5.0,0.0,0.126,0.000
1,Afghanistan,5.0,0.0,0.126,0.000
2,Afghanistan,5.0,0.0,0.126,0.000
3,Afghanistan,5.0,0.0,0.126,0.000
4,Afghanistan,5.0,0.0,0.126,0.000
...,...,...,...,...,...
138212,Zimbabwe,561.0,6.0,37.172,0.398
138213,Zimbabwe,567.0,6.0,37.569,0.398
138214,Zimbabwe,567.0,6.0,37.569,0.398
138215,Zimbabwe,574.0,7.0,38.033,0.464


# 이상값의 정의
- 다소 주관적이며(Somewhat Subjective), 특정 분포의 중심경향성, 퍼진 정도와 형태에 따라 밀접한 관련이 있다.
  + 평균에서 표준편차보다 몇 배 더 떨어져 있다거나, 즉, 정규분포를 이루고 있지 않을 때
  + 왜도 또는 첨도가 발생할 때
- 균등분포(Uniform Distribution)는, 발생할 확률이 모두 같다.
  + 만약, 확진자수가 최소 1부터 최대 10,000,000까지 균등하게 분포한다면, 어떤 값도 이상값으로 고려하지 않는다.
- 이상값을 파악하려면, 반드시, 각 변수의 분포를 먼저 이해해야 한다.

In [62]:
case_vars = ["location", "total_cases", "total_deaths", "total_cases_per_million", "total_deaths_per_million"]
demo_vars = ["population", "population_density", "median_age", "gdp_per_capita", "hospital_beds_per_thousand"]

outlier_df = df.loc[df['date'].between("2020-02-24", "2020-04-30")]
outlier_df = outlier_df[demo_vars + case_vars]

- describe() 함수를 통해서 수치 데이터의 분포를 확인한다. 

In [64]:
outlier_case_df = outlier_df.loc[:, case_vars]
outlier_case_df = outlier_case_df.drop_duplicates()
outlier_case_df.describe()

,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million
count,8.414000e+03,6388.000000,8402.000000,6376.000000
mean,3.783374e+04,3386.901691,419.007538,26.737916
std,1.987200e+05,17065.420620,1122.243141,94.933415
min,1.000000e+00,1.000000,0.001000,0.001000
25%,6.900000e+01,5.000000,6.205500,0.353000
50%,4.820000e+02,23.000000,46.831000,1.764500
75%,3.897000e+03,206.000000,269.339250,9.798500
max,3.281736e+06,240996.000000,16553.955000,1205.528000


- 백분위수로 quantile로 데이터를 표시한다.

In [65]:
outlier_case_df.quantile(np.arange(0.0, 1.1, 0.1))

,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million
0.0,1.0,1.0,0.0010,0.0010
0.1,13.0,1.0,0.9162,0.0880
0.2,44.0,3.0,3.4770,0.2470
0.3,104.0,6.0,10.0274,0.4870
0.4,237.0,10.0,23.4426,0.9200
0.5,482.0,23.0,46.8310,1.7645
0.6,1009.0,50.0,92.9934,3.0130
0.7,2272.1,122.9,190.4990,6.0680
0.8,6897.6,421.6,433.0640,15.7140
0.9,37837.7,3433.8,1184.3308,51.3010


In [71]:
outlier_case_df.shape

(8442, 5)

- 연속형 변수의 이상값 범위는 1사분위-3사분위의 거리를 재는 것이다. 그 거리가 사분위범위의 1.5배를 넘으면 그 값을 이상값으로 간주한다.

In [72]:
thirdq = outlier_case_df["total_cases"].quantile(0.75)
firstq = outlier_case_df["total_cases"].quantile(0.25)
interquantile_range = 1.5 * (thirdq - firstq)
outlier_high = interquantile_range + thirdq
outlier_low = firstq - interquantile_range

print(outlier_low, outlier_high, sep = " <---> ")

-5673.0 <---> 9639.0


- 이상치를 제거한 데이터를 가져온다. 

In [82]:
remove_out_lier_df = outlier_case_df.loc[~(outlier_case_df["total_cases"] > outlier_high) | (outlier_case_df["total_cases"] < outlier_low)]
print(remove_out_lier_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7015 entries, 0 to 153009
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   location                  7015 non-null   object 
 1   total_cases               6987 non-null   float64
 2   total_deaths              4961 non-null   float64
 3   total_cases_per_million   6975 non-null   float64
 4   total_deaths_per_million  4949 non-null   float64
dtypes: float64(4), object(1)
memory usage: 328.8+ KB
None


In [85]:
remove_out_lier_df = remove_out_lier_df.reset_index(drop=True)
remove_out_lier_df

,location,total_cases,total_deaths,total_cases_per_million,total_deaths_per_million
0,Afghanistan,5.0,NaN,0.126,NaN
1,Afghanistan,8.0,NaN,0.201,NaN
2,Afghanistan,11.0,NaN,0.276,NaN
3,Afghanistan,14.0,NaN,0.351,NaN
4,Afghanistan,20.0,NaN,0.502,NaN
...,...,...,...,...,...
7010,Zimbabwe,28.0,4.0,1.855,0.265
7011,Zimbabwe,29.0,4.0,1.922,0.265
7012,Zimbabwe,31.0,4.0,2.054,0.265
7013,Zimbabwe,32.0,4.0,2.120,0.265


# NumPy와 Pandas와의 조합

In [86]:
import pandas as pd 
import numpy as np

df = pd.DataFrame({
    '과일': ['사과', '자두', '사과', '자두'],
    '판매사': ['A', 'A', 'B', 'B'],
    '판매무게 (kg)': [10,20,30,40],
    '판매별점': [4, 3, 5, 3.5]
})

A_price = pd.DataFrame({
    '과일': ['사과', '바나나', '자두', '파인애플'],
    '가격 (kg)': [7000, 5000, 5000, 6000]
})

B_price = pd.DataFrame({
    '과일': ['사과', '바나나', '자두', '파인애플'],
    '가격 (kg)': [6500, 5500, 6000, 5000]
})

- 기존 df에 각 품목에 맞는 것만 가져와서 kg당 가격을 추가하는 코드를 짜본다. 
- 먼저 `fruit` 함수를 인덱스로 추가한다. 

In [87]:
df = df.set_index("과일")
A_price = A_price.set_index("과일")
B_price = B_price.set_index("과일")

- np.where 함수를 적용하여 각 판매사에 맞는 kg당 가격을 추가한다. 

In [88]:
df['가격 (kg)'] = np.where(
    df['판매사'] == 'A', 
    A_price.loc[df.index]['가격 (kg)'], 
    B_price.loc[df.index]['가격 (kg)']
)

df

,판매사,판매무게 (kg),판매별점,가격 (kg)
과일,,,,
사과,A,10,4.0,7000
자두,A,20,3.0,5000
사과,B,30,5.0,6500
자두,B,40,3.5,6000


- 이번에는 판매사가 3개일 때 경우의 수를 생각해본다. 

In [89]:
import pandas as pd 
import numpy as np

df = pd.DataFrame({
    '과일': ['사과', '자두', '사과', '자두', '사과', '자두'],
    '판매사': ['A', 'A', 'B', 'B', 'C', 'C'],
    '판매무게 (kg)': [10,20,30,40, 15, 20],
    '판매별점': [4, 3, 5, 3.5, 4, 5]
})

df_price = pd.DataFrame({
    '과일': ['사과', '바나나', '자두', '파인애플'],
    'A' : [7000, 5000, 5000, 6000], 
    'B' : [6500, 5500, 6000, 5000], 
    'C' : [2000, 3000, 3000, 4000]
})

df = df.set_index("과일")
df

,판매사,판매무게 (kg),판매별점
과일,,,
사과,A,10,4.0
자두,A,20,3.0
사과,B,30,5.0
자두,B,40,3.5
사과,C,15,4.0
자두,C,20,5.0


In [90]:
df_price = df_price.set_index('과일')
df_price

,A,B,C
과일,,,
사과,7000,6500,2000
바나나,5000,5500,3000
자두,5000,6000,3000
파인애플,6000,5000,4000


In [91]:
args = df_price.loc[df.index]
args

,A,B,C
과일,,,
사과,7000,6500,2000
자두,5000,6000,3000
사과,7000,6500,2000
자두,5000,6000,3000
사과,7000,6500,2000
자두,5000,6000,3000


In [92]:
conds = [
    df['판매사'] == 'A', 
    df['판매사'] == 'B', 
    df['판매사'] == 'C', 
]

choices = [
    args['A'], 
    args['B'], 
    args['C']
]
choices

[과일
 사과    7000
 자두    5000
 사과    7000
 자두    5000
 사과    7000
 자두    5000
 Name: A, dtype: int64, 과일
 사과    6500
 자두    6000
 사과    6500
 자두    6000
 사과    6500
 자두    6000
 Name: B, dtype: int64, 과일
 사과    2000
 자두    3000
 사과    2000
 자두    3000
 사과    2000
 자두    3000
 Name: C, dtype: int64]

In [93]:
df['price (kg)'] = np.select(conds, choices)
df

,판매사,판매무게 (kg),판매별점,price (kg)
과일,,,,
사과,A,10,4.0,7000
자두,A,20,3.0,5000
사과,B,30,5.0,6500
자두,B,40,3.5,6000
사과,C,15,4.0,2000
자두,C,20,5.0,3000


# NumPy 함수 활용한 변수 선택하기 

In [94]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153630 entries, 0 to 153629
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    153630 non-null  object 
 1   continent                                   144376 non-null  object 
 2   location                                    153630 non-null  object 
 3   date                                        153630 non-null  object 
 4   total_cases                                 150978 non-null  float64
 5   new_cases                                   150903 non-null  float64
 6   new_cases_smoothed                          149752 non-null  float64
 7   total_deaths                                133709 non-null  float64
 8   new_deaths                                  133870 non-null  float64
 9   new_deaths_smoothed                         133738 non-null  float64
 

- 참조: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html

In [97]:
df_objects_only = df.select_dtypes(include='object')
df_objects_only

,iso_code,continent,location,date,tests_units
0,AFG,Asia,Afghanistan,2020-02-24,NaN
1,AFG,Asia,Afghanistan,2020-02-25,NaN
2,AFG,Asia,Afghanistan,2020-02-26,NaN
3,AFG,Asia,Afghanistan,2020-02-27,NaN
4,AFG,Asia,Afghanistan,2020-02-28,NaN
...,...,...,...,...,...
153625,ZWE,Africa,Zimbabwe,2022-01-06,tests performed
153626,ZWE,Africa,Zimbabwe,2022-01-07,tests performed
153627,ZWE,Africa,Zimbabwe,2022-01-08,NaN
153628,ZWE,Africa,Zimbabwe,2022-01-09,NaN


In [95]:
df_numerics_only = df.select_dtypes(include=np.number)
df_numerics_only

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153625,220178.0,1121.0,1207.143,5108.0,16.0,15.857,14588.889,74.277,79.985,338.454,1.060,1.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6048.0,1740966.0,115.356,0.401,5698.0,0.378,0.2119,4.7,7323933.0,4154155.0,3169778.0,NaN,13877.0,11002.0,48.53,27.53,21.00,NaN,729.0,5101.0,0.034,52.78,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
153626,221282.0,1104.0,1146.286,5136.0,28.0,18.857,14662.039,73.151,75.952,340.309,1.855,1.249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6723.0,1747689.0,115.801,0.445,5780.0,0.383,0.1983,5.0,7338669.0,4160978.0,3177691.0,NaN,14736.0,11303.0,48.63,27.57,21.06,NaN,749.0,5229.0,0.035,52.78,15092171.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
153627,221918.0,636.0,1100.571,5148.0,12.0,18.714,14704.180,42.141,72.923,341.104,0.795,1.240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7346907.